In [30]:
import numpy as np
import pandas as pd
import scipy
from gym import error, spaces, utils
from collections import OrderedDict
from sklearn import preprocessing
from sklearn.externals import joblib as jb
import random
import time

## Let's set up everything for the Advanced Attack...

In [31]:
important_features=['source_app_bytes','remote_app_bytes','tcp_packets','remote_app_packets','vulume_bytes','source_app_packets']
n_states=len(important_features)
n_actions=scipy.math.factorial(n_states)
e = 0.9 # greedy factor
a= 0.1  # learning rate
g= 0.9 # discount factor
max_episodes = 3 # maximum episodes
fresh_time= 0.3 # fresh time for one move
malicious_data=pd.read_csv("../Datasets/Colcom2018CleanedScaled.csv",header=0,sep=';')
adversarial_data=pd.read_csv("../Datasets/Colcom2018CleanedScaled.csv",header=0,sep=';')

In [32]:
adversarial_data.drop(adversarial_data.index, inplace=True)
adversarial_data= pd.DataFrame(np.zeros((1,len(list(adversarial_data.columns.values)))), columns=list(adversarial_data.columns.values)).append(adversarial_data, ignore_index=True).copy()
adversarial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
tcp_packets           1 non-null float64
dist_port_tcp         1 non-null float64
external_ips          1 non-null float64
vulume_bytes          1 non-null float64
udp_packets           1 non-null float64
source_app_packets    1 non-null float64
remote_app_packets    1 non-null float64
 source_app_bytes     1 non-null float64
remote_app_bytes      1 non-null float64
dns_query_times       1 non-null float64
type                  1 non-null float64
dtypes: float64(11)
memory usage: 168.0 bytes


In [33]:
malicious_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7832 entries, 0 to 7831
Data columns (total 11 columns):
tcp_packets           7832 non-null float64
dist_port_tcp         7832 non-null float64
external_ips          7832 non-null float64
vulume_bytes          7832 non-null float64
udp_packets           7832 non-null float64
source_app_packets    7832 non-null float64
remote_app_packets    7832 non-null float64
 source_app_bytes     7832 non-null float64
remote_app_bytes      7832 non-null float64
dns_query_times       7832 non-null float64
type                  7832 non-null object
dtypes: float64(10), object(1)
memory usage: 673.1+ KB


In [34]:
adversarial_data= adversarial_data.drop('type', axis=1).copy()

In [35]:
malicious_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7832 entries, 0 to 7831
Data columns (total 11 columns):
tcp_packets           7832 non-null float64
dist_port_tcp         7832 non-null float64
external_ips          7832 non-null float64
vulume_bytes          7832 non-null float64
udp_packets           7832 non-null float64
source_app_packets    7832 non-null float64
remote_app_packets    7832 non-null float64
 source_app_bytes     7832 non-null float64
remote_app_bytes      7832 non-null float64
dns_query_times       7832 non-null float64
type                  7832 non-null object
dtypes: float64(10), object(1)
memory usage: 673.1+ KB


In [36]:
adversarial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
tcp_packets           1 non-null float64
dist_port_tcp         1 non-null float64
external_ips          1 non-null float64
vulume_bytes          1 non-null float64
udp_packets           1 non-null float64
source_app_packets    1 non-null float64
remote_app_packets    1 non-null float64
 source_app_bytes     1 non-null float64
remote_app_bytes      1 non-null float64
dns_query_times       1 non-null float64
dtypes: float64(10)
memory usage: 160.0 bytes


In [37]:
adversarial_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
malicious_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,0.127907,6.0,0.333333,0.036782,0.0,0.100000,0.105882,0.041652,0.029865,0.00,benign
1,1.069767,0.0,2.333333,1.786894,0.0,1.088889,0.976471,0.897275,1.787684,2.00,benign
2,1.988372,0.0,1.333333,1.843764,0.0,1.944444,2.235294,6.465984,1.831939,1.50,benign
3,-0.220930,0.0,-0.333333,-0.233015,0.0,-0.255556,-0.211765,-0.131552,-0.245310,-0.50,benign
4,-0.220930,0.0,-0.333333,-0.233640,0.0,-0.255556,-0.211765,-0.131552,-0.245919,-0.50,benign
5,0.337209,54.0,0.333333,0.139541,0.0,0.366667,0.352941,0.056096,0.167148,1.50,benign
6,-0.220930,0.0,-0.333333,-0.233015,0.0,-0.255556,-0.211765,-0.131552,-0.245310,-0.50,benign
7,-0.220930,0.0,-0.333333,-0.209356,0.0,-0.255556,-0.211765,-0.140695,-0.223314,-0.50,benign
8,-0.220930,0.0,-0.333333,-0.233104,0.0,-0.255556,-0.211765,-0.131552,-0.245397,-0.50,benign
9,1.988372,0.0,0.333333,1.125882,0.0,1.888889,2.823529,11.992050,1.096659,0.25,benign


In [39]:
malicious_data= malicious_data[malicious_data.type=='malicious'].copy()

In [40]:
malicious_data=malicious_data.reset_index(drop=True)

In [41]:
malicious_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.159145,-0.323559,-0.50,malicious
1,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.159185,-0.323646,-0.50,malicious
2,-0.174419,0.0,-0.333333,-0.229087,0.0,-0.200000,-0.164706,-0.092024,-0.235921,-0.25,malicious
3,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158578,-0.323299,-0.50,malicious
4,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.211765,-0.138673,-0.323559,-0.50,malicious
5,0.348837,11.0,0.333333,0.396572,0.0,0.333333,0.317647,0.458823,0.394244,0.50,malicious
6,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158578,-0.323299,-0.50,malicious
7,-0.011628,0.0,-0.333333,-0.085082,0.0,-0.011111,0.070588,0.196205,-0.072467,0.50,malicious
8,-0.220930,0.0,-0.333333,-0.272744,0.0,-0.244444,-0.200000,-0.129327,-0.277306,-0.25,malicious
9,-0.186047,9.0,-0.333333,-0.252924,0.0,-0.200000,-0.247059,-0.148302,-0.252614,0.00,malicious


In [42]:
malicious_data = malicious_data.drop('type', axis=1).copy()

In [43]:
malicious_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.159145,-0.323559,-0.50
1,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.159185,-0.323646,-0.50
2,-0.174419,0.0,-0.333333,-0.229087,0.0,-0.200000,-0.164706,-0.092024,-0.235921,-0.25
3,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158578,-0.323299,-0.50
4,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.211765,-0.138673,-0.323559,-0.50
5,0.348837,11.0,0.333333,0.396572,0.0,0.333333,0.317647,0.458823,0.394244,0.50
6,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158578,-0.323299,-0.50
7,-0.011628,0.0,-0.333333,-0.085082,0.0,-0.011111,0.070588,0.196205,-0.072467,0.50
8,-0.220930,0.0,-0.333333,-0.272744,0.0,-0.244444,-0.200000,-0.129327,-0.277306,-0.25
9,-0.186047,9.0,-0.333333,-0.252924,0.0,-0.200000,-0.247059,-0.148302,-0.252614,0.00


In [44]:
malicious_data.shape

(3141, 10)

In [45]:
adversarial_data.shape

(1, 10)

In [46]:
adversarial_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
adversarial_data= malicious_data.sample(n=500).copy()

In [48]:
adversarial_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
1048,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.311111,-0.258824,-0.153845,-0.317386,-0.25
248,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.311111,-0.258824,-0.153764,-0.317734,-0.25
1875,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158740,-0.323646,-0.50
2625,0.267442,0.0,0.333333,0.501384,0.0,0.333333,0.600000,1.454393,0.520834,1.25
264,-0.151163,12.0,0.666667,-0.233104,0.0,-0.188889,-0.270588,-0.156029,-0.246267,-0.50
829,10.244186,0.0,1.333333,18.849924,0.0,9.811111,9.670588,4.253171,18.375334,1.00
2444,0.127907,0.0,0.000000,0.217570,0.0,0.100000,0.105882,-0.004268,0.206273,0.00
1618,-0.162791,0.0,0.000000,-0.222302,0.0,-0.188889,-0.211765,-0.126009,-0.229139,-0.25
263,-0.151163,12.0,0.666667,-0.233104,0.0,-0.188889,-0.270588,-0.156029,-0.246267,-0.50
1966,-0.244186,0.0,-0.333333,-0.285957,0.0,-0.244444,-0.235294,-0.140695,-0.278610,0.25


## Let's build the Q-table

In [49]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
    np.zeros((actions, n_states)), # q_table initial values
    columns = important_features # Important features
    )
    return table

build_q_table(n_states, n_actions).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
source_app_bytes      720 non-null float64
remote_app_bytes      720 non-null float64
tcp_packets           720 non-null float64
remote_app_packets    720 non-null float64
vulume_bytes          720 non-null float64
source_app_packets    720 non-null float64
dtypes: float64(6)
memory usage: 33.8 KB


## Let's choose the action

In [50]:
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > e) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(important_features)
    else:   # act greedy
        action_name = state_actions.idxmax()   # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name

str(choose_action(4,build_q_table(n_states, n_actions)))

'vulume_bytes'

## Alright, let's upload the best model to attack

In [51]:
model=jb.load('rfDefense.sav')
print(model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=None,
            oob_score=False, random_state=45, verbose=0, warm_start=False)


In [52]:
model.predict(adversarial_data.iloc[0:7,:])

array(['malicious', 'malicious', 'malicious', 'malicious', 'malicious',
       'malicious', 'malicious'], dtype=object)

## Feedback...

In [53]:
def get_env_feedback(S, A, r):
    # This is how agent will interact with the environment
    pred = model.predict(A.iloc[r].values.reshape(1,-1))
    if pred == 'benign':    # it is detected as benign
        if S == n_states - 2:
            R = 2
            S_ = 'terminal'            
        else:
            S_ = S + 1
            R = 1
    else:
        R = 0
        if S == 0:
            S_ = S  # reach the defense
        else:
            S_ = S - 1
    return S_, R   
    
#get_env_feedback(1, complete_permissions)

In [54]:
def update_env(S, episode, step_counter, r):
    # This is how environment be updated
    env_list = ['-']*(n_states-1) + ['|']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s. Row %s.' % (episode+1, step_counter, r)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        if n_states - S == 2:
            env_list[S] = 'B'
            interaction = ''.join(env_list)
            print('\r{}'.format(interaction), end='')
            time.sleep(fresh_time)
        else:            
            env_list[S] = 'M'
            interaction = ''.join(env_list)
            print('\r{}'.format(interaction), end='')
            time.sleep(fresh_time)

In [55]:
def rl():    
    q_table = build_q_table(n_states, n_actions)
    adv_data = adversarial_data.copy()
    for row in range(adv_data.shape[0]):
        for episode in range(max_episodes):
            step_counter = 0
            S = 0
            is_terminated = False
            update_env(S, episode, step_counter, row)
            while not is_terminated:   
                for i in range(0,3):
                    A = choose_action(S, q_table)
                    if A == 'source_app_bytes':
                        adv_data.iloc[row,7] = round(random.uniform(-0.164688,715.616633),6)
                    elif A == 'remote_app_bytes':
                        adv_data.iloc[row,8] = round(random.uniform(-0.324081,106.407677),6)
                    elif A == 'tcp_packets':
                        adv_data.iloc[row,0] = round(random.uniform(-0.290698,133.441860),6)
                    elif A == 'remote_app_packets':
                        adv_data.iloc[row,6] = round(random.uniform(-0.282353,147.976471),6)  
                    elif A == 'vulume_bytes':
                        adv_data.iloc[row,3] = round(random.uniform(-0.312383,109.259173),6)
                    else:
                        adv_data.iloc[row,5] = round(random.uniform(-0.322222,127.488889),6)

                S_, R = get_env_feedback(S, adv_data, row)  # take action & get next state and reward
                q_predict = q_table.loc[S, A]
                if S_ != 'terminal':
                    q_target = R + g * q_table.iloc[S_, :].max()   # next state is not terminal
                else:
                    q_target = R     # next state is terminal
                    is_terminated = True    # terminate this episode  

                q_table.loc[S, A] += a * (q_target - q_predict)  # update
                S = S_  # move to next state  
                update_env(S, episode, step_counter+1, row)
                step_counter += 1
    return q_table, adv_data

In [56]:
# let's start it 
q_table, adv_data = rl()
print('\r\nQ-table:\n')
print(q_table)

                                499..
Q-table:

     source_app_bytes  remote_app_bytes  tcp_packets  remote_app_packets  \
0            4.337266          4.309849     4.365914            4.472672   
1            3.251161          3.411957     3.625106            3.258835   
2            3.254083          3.240275     2.923339            3.315432   
3            2.731354          2.620044     2.800000            2.654352   
4            2.000000          1.822741     1.781162            1.856420   
5            0.000000          0.000000     0.000000            0.000000   
6            0.000000          0.000000     0.000000            0.000000   
7            0.000000          0.000000     0.000000            0.000000   
8            0.000000          0.000000     0.000000            0.000000   
9            0.000000          0.000000     0.000000            0.000000   
10           0.000000          0.000000     0.000000            0.000000   
11           0.000000          0.000000 

In [57]:
#before the attack
adversarial_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
1048,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.311111,-0.258824,-0.153845,-0.317386,-0.25
248,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.311111,-0.258824,-0.153764,-0.317734,-0.25
1875,-0.290698,0.0,-0.666667,-0.312383,0.0,-0.322222,-0.270588,-0.158740,-0.323646,-0.50
2625,0.267442,0.0,0.333333,0.501384,0.0,0.333333,0.600000,1.454393,0.520834,1.25
264,-0.151163,12.0,0.666667,-0.233104,0.0,-0.188889,-0.270588,-0.156029,-0.246267,-0.50
829,10.244186,0.0,1.333333,18.849924,0.0,9.811111,9.670588,4.253171,18.375334,1.00
2444,0.127907,0.0,0.000000,0.217570,0.0,0.100000,0.105882,-0.004268,0.206273,0.00
1618,-0.162791,0.0,0.000000,-0.222302,0.0,-0.188889,-0.211765,-0.126009,-0.229139,-0.25
263,-0.151163,12.0,0.666667,-0.233104,0.0,-0.188889,-0.270588,-0.156029,-0.246267,-0.50
1966,-0.244186,0.0,-0.333333,-0.285957,0.0,-0.244444,-0.235294,-0.140695,-0.278610,0.25


In [58]:
#After the attack
adv_data

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
1048,5.927622,0.0,-0.666667,39.884274,0.0,103.199390,0.700097,437.493903,1.994524,-0.25
248,22.138217,0.0,-0.666667,-0.312383,0.0,31.467176,-0.258824,625.873298,24.786021,-0.25
1875,104.151100,0.0,-0.666667,107.834553,0.0,35.351266,-0.270588,326.570370,60.857192,-0.50
2625,105.675003,0.0,0.333333,107.001677,0.0,64.329648,78.899647,651.740906,0.520834,1.25
264,133.315081,12.0,0.666667,-0.233104,0.0,49.437327,107.038216,194.724991,87.570171,-0.50
829,22.122525,0.0,1.333333,18.849924,0.0,108.625747,5.160303,424.814396,18.375334,1.00
2444,60.937162,0.0,0.000000,84.464224,0.0,77.185314,0.105882,220.388790,0.206273,0.00
1618,51.344471,0.0,0.000000,-0.222302,0.0,85.450092,-0.211765,710.062010,7.298407,-0.25
263,17.316595,12.0,0.666667,57.195326,0.0,96.196933,126.243125,290.894504,-0.246267,-0.50
1966,108.748722,0.0,-0.333333,41.458794,0.0,7.063031,86.855975,417.421244,12.842077,0.25


In [59]:
count=0
for i in range(adv_data.shape[0]):
    if model.predict(adv_data.iloc[i].values.reshape(1,-1))=='benign':
        print("Hacked! :)")
        count+=1
        print(count)
    else:
        print("Still Malware :(")

Hacked! :)
1
Hacked! :)
2
Hacked! :)
3
Hacked! :)
4
Hacked! :)
5
Hacked! :)
6
Hacked! :)
7
Hacked! :)
8
Hacked! :)
9
Hacked! :)
10
Hacked! :)
11
Hacked! :)
12
Hacked! :)
13
Hacked! :)
14
Hacked! :)
15
Hacked! :)
16
Hacked! :)
17
Hacked! :)
18
Hacked! :)
19
Hacked! :)
20
Hacked! :)
21
Hacked! :)
22
Hacked! :)
23
Hacked! :)
24
Hacked! :)
25
Hacked! :)
26
Hacked! :)
27
Hacked! :)
28
Hacked! :)
29
Hacked! :)
30
Hacked! :)
31
Hacked! :)
32
Hacked! :)
33
Hacked! :)
34
Hacked! :)
35
Hacked! :)
36
Hacked! :)
37
Hacked! :)
38
Hacked! :)
39
Hacked! :)
40
Hacked! :)
41
Hacked! :)
42
Hacked! :)
43
Hacked! :)
44
Hacked! :)
45
Hacked! :)
46
Hacked! :)
47
Hacked! :)
48
Hacked! :)
49
Hacked! :)
50
Hacked! :)
51
Hacked! :)
52
Hacked! :)
53
Hacked! :)
54
Hacked! :)
55
Hacked! :)
56
Hacked! :)
57
Hacked! :)
58
Hacked! :)
59
Hacked! :)
60
Hacked! :)
61
Hacked! :)
62
Hacked! :)
63
Hacked! :)
64
Hacked! :)
65
Hacked! :)
66
Hacked! :)
67
Hacked! :)
68
Hacked! :)
69
Hacked! :)
70
Hacked! :)
71
Hacked! :)
72
H

## Let's export the Q-table

In [60]:
jb.dump(q_table, 'Q-table.sav')

['Q-table.sav']

In [61]:
adv_data.to_csv('../Datasets/Colcom2018AdversarialData.csv',sep=';',index=False)